In [25]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import random

In [26]:

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM endereco"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [27]:
# Anonimização Rua



def anonimizar_rua(senha, chave):
    senha = str(senha)
    chave = str(chave).lower()
    chave_anonimizacao = ''
    i = 0
    for l in senha:
          # deslocar com base na letra da chave (cifra de Vigenère adaptada para anonimização)
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        #mantem dentro do intervalo de caracteres imprimíveis, no caso 32–126
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        chave_anonimizacao += nova_letra
        i += 1
    return chave_anonimizacao

# gera uma chave aleatória para a anonimização
chave_anonimizacao = ''.join(chr(random.randint(97, 122)) for _ in range(10))
df_anonimizado["rua"] = df_anonimizado["rua"].apply(lambda x: anonimizar_rua(x, chave_anonimizacao))
# mostra os resultados
print("==== Dados anonimizados ====")
display(df_anonimizado[["rua"]])

==== Dados anonimizados ====


,rua
0,I(itlyy*Ve}}mywv
1,I(itlyy*Lezze&O~&k
2,Z'e&F%'}kppvmxr5[kxv-!
3,Z'e&gv8Wuskr
4,"Z'e&gv,*Lpw$iy"
5,"Z'e&wz,~k6"


In [28]:
from IPython.display import display

def anonimizar_numero(numero, chave):
    numero = str(numero)
    chave = str(chave).lower()
    i = 0
    for l in numero:
        return "*****"

# define a chave fixa antes de usar
chave = 5

df_anonimizado["numero"] = df_anonimizado["numero"].apply(lambda x: anonimizar_numero(x, chave))

print("==== Dados anonimizados ====")
display(df_anonimizado[["numero"]])



==== Dados anonimizados ====


,numero
0,*****
1,*****
2,*****
3,*****
4,*****
5,*****


In [29]:
#pais pseudonimizado

def pseudonimizar_pais(pais):
    # Remove espaços extras
    pais= pais.strip()

    uf = pais[:2].upper()
    return f"{uf}"

# aplica 
df_anonimizado["pais"] = df_anonimizado["pais"].apply(pseudonimizar_pais)

# Mostra resultados
print("==== Dados anonimizados ====")
display(df_anonimizado[["pais"]])



==== Dados anonimizados ====


,pais
0,BR
1,BR
2,BR
3,BR
4,BR
5,PA


In [30]:
# Bairro anonimizado

def anonimizar_bairro(bairro, chave):
    senha = str(bairro)
    chave = str(chave).lower()
    senha_cripto = ''
    i = 0
    for l in senha:
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        senha_cripto += nova_letra
        i += 1
    return senha_cripto


# aplica a criptografia no campo bairro 
chave = "minhachave"
df_anonimizado["bairro"] = df_anonimizado["bairro"].apply(lambda x: anonimizar_bairro(x, chave))
# mostra os resultados
print("==== Dados anonimizados ====")
display(df_anonimizado[["bairro"]])



==== Dados anonimizados ====


,bairro
0,Nmyh Xps*e
1,\q{oekyo)
2,bqyh Ehr(){
3,Yw|ja
4,Om{{rq
5,Nivyrq'tzw!m?


In [31]:
def pseudonimizar_cep(cep):

    # garante string caso no banco nao seja
    cep = str(cep)
    
    #divide o em duas partes, antes e depois do hífen
    if "-" in cep:
        parte_local, sufixo = cep.split("-")
    else:
        parte_local, sufixo = cep, ""
    
    # substitui os carácteres por *
    if len(parte_local) > 5:
        parte_local_pseudo = parte_local[:4] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    
    # junta de volta para manter o padrão do cep
    if sufixo:
        return parte_local_pseudo + "-" + sufixo
    else:
        return parte_local_pseudo
      

# aplica pseudonimizacao
df_anonimizado["cep"] = df_anonimizado["cep"].apply(pseudonimizar_cep)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cep"]])


==== Dados anonimizados ====


,cep
0,0131****
1,0540****
2,0826****
3,0316****
4,1234****
5,1234****


In [32]:
display(df_anonimizado)

,id,cep,rua,numero,bairro,cidade,estado,pais
0,1,0131****,I(itlyy*Ve}}mywv,*****,Nmyh Xps*e,São Paulo,SP,BR
1,3,0540****,I(itlyy*Lezze&O~&k,*****,\q{oekyo),São Paulo,SP,BR
2,4,0826****,Z'e&F%'}kppvmxr5[kxv-!,*****,bqyh Ehr(){,São Paulo,SP,BR
3,5,0316****,Z'e&gv8Wuskr,*****,Yw|ja,São Paulo,SP,BR
4,2,1234****,"Z'e&gv,*Lpw$iy",*****,Om{{rq,São Paulo,SP,BR
5,6,1234****,"Z'e&wz,~k6",*****,Nivyrq'tzw!m?,Cidade teste2,Estado Teste2,PA


In [33]:
# insere do banco de anonimizacao/pseudominizacao
from sqlalchemy import text

with engine_saida.begin() as conn:
    # por depender de outras tabelas, limpa a tabela sem quebrar FK e reinicia o ID
    conn.execute(text("TRUNCATE TABLE endereco RESTART IDENTITY CASCADE"))

df_anonimizado.to_sql(name='endereco', con=engine_saida, if_exists='append', index=False)



6

In [24]:
#mandar para o csv
dados_originais_df.to_csv('original_endereco.csv')
df_anonimizado.to_csv('pseudo_endereco.csv')